In [8]:
import jn_setup
from simulation_procedure import model
from tools.paths import *
import pandas as pd, pyabc, hydroeval as he, numpy as np, uuid

In [9]:
"""
Priors. Get the values from that csv. Just for SWMM at first.
"""
swmm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

'''
Link up with the vvwm priors and make one big list with 36 params
'''
vvwm_ranges = pd.read_csv(os.path.join(master_path, "lhs_param_ranges_vvwm.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

priors = param_ranges.to_dict("index")

# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})

In [10]:
prior

<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'Rough', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'rflat', 'sol', 'sused'>

# Make the .new object
### 1. Import observed data

In [11]:
# import it again to make inspect it
# specifically for TEST mode!
with open(os.path.join(main_path, 'master_test','test_obs_data.txt'),'r') as read_file:
    obs_dict = eval(read_file.read())
# obs_dict

### 2. Initialize dask client for dask distributed sampler

In [12]:
from dask.distributed import Client#, LocalCluster
# cluster = LocalCluster()#n_workers=(90/2), threads_per_worker = 2)  # Set for 96 vCPU compute instance
# client = Client(cluster)#,timeout=400)

# make it simpler
# if __name__ == "__main__":
client = Client()

sampler = pyabc.sampler.DaskDistributedSampler(dask_client = client)

C:\Users\jstelman\Anaconda3\envs\swmm1\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57786 instead
  warnings.warn(


### 3. Set up a sqlite db directory

In [13]:
# Initialize a new ABC inference run
dbid = uuid.uuid4().hex[0:8]
print(dbid)
database_dir = os.path.join(temp_path, 'results_db')  
if not os.path.exists(database_dir):
    os.mkdir(database_dir)
db_path = ("sqlite:///" +
           os.path.join(database_dir, "test_pyabc_" + dbid + ".db"))

5e43a2d4


### 4. Defining a Distance function

We need to refactor the NSE distance function using the pyabc.Distance class.
We will need the hydroeval library and the pyabc.SimpleFunctionDistance to do this

In [14]:
# make a file to hold onto these NSEs for our own record
with open(dbid+".txt","w") as nse_file:
    nse_file.write("NSEs\n")

In [15]:
def nse(x, x_0):
    nse = he.evaluator(he.nse, 
                       simulation_s = np.array(list(x.values())), 
                       evaluation = np.array(list(x_0.values())))[0]
    # print("simulation_s ",np.array(list(x.values())))
    # print("evaluation ", np.array(list(x_0.values())))
    print("nse ", nse)
    # make record
    with open(dbid+".txt","a") as nse_file:
        nse_file.write(str(nse)+"\n")
    return nse
NSE = pyabc.SimpleFunctionDistance(fun=nse)


### 5. Define ABCSMC object

In [16]:
abc = pyabc.ABCSMC(model, prior, #distance_function = hydroeval.nse, 
                   population_size = pyabc.AdaptivePopulationSize(40, max_population_size = 40), # just to shorten the run
                   sampler = sampler,
                   distance_function = NSE)
                   #distance_function = pyabc.PNormDistance(p=2))

In [17]:
abc

### 6. Initialize a new abc run

In [18]:
abc.new(db_path, obs_dict)

In [ ]:
history = abc.run(max_nr_populations=1, minimum_epsilon=0.2)

nse  -1.8535425267842862e+24
nse  -2.91340887162716e+22
nse  -8.477655688998076e+22
nse  -3.4499030740844673e+24
nse  -1.837370633865347e+22
nse  -1.9494667478024945e+26
nse  -1.4167642772328968e+25
nse  -5.108075720863359e+22
nse  -1.1534858283879951e+24
nse  -6.222612720192415e+23
nse  -8.072120435365622e+23
nse  -5.29682754397557e+22
nse  -8.889711829153913e+23
nse  -2.587632998955404e+23
nse  -5.463889584980147e+22
nse  -3.025954500699361e+23
nse  -1.3982018800740644e+23
nse  -3.152577905645993e+25
nse  -1.1489035051197777e+24
nse  -1.0086346483323143e+23
nse  -1.7861012901054358e+23
nse  -9.198722445260154e+22
nse  -1.2920116597084387e+23
nse  -4.316200295260966e+23
nse  -1.7003676734489177e+24
nse  -2.0646497415521854e+25
nse  -2.8723863826781425e+25
nse  -5.012723750817788e+25
nse  -1.2645545880282536e+23
nse  -2.6074245310214832e+25
nse  -1.54077895431479e+24
nse  -2.2256538284549376e+23
nse  -1.5484929955716697e+25
nse  -8.933022755172301e+24
nse  -2.2152759162174488e+23
nse  